In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
headers = {
    'accept': 'application/json',
}

params = {
    'siglaUf': 'AL',
    'dataInicio': '2019-01-31',
    'dataFim': '2023-01-31',
    'ordem': 'ASC',
    'ordenarPor': 'nome',
}

response = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados', params=params, headers=headers)

In [ ]:
dados = response.json()
dados = dados['dados']
df_dep = pd.DataFrame(dados)
df_dep = df_dep[['id', 'nome', 'siglaPartido', 'siglaUf']]
df_dep['nome'] = df_dep['nome'].apply(lambda x: x.upper())
df_dep = df_dep.drop_duplicates(subset=['nome'])

In [5]:
temp_df = []

for dep in df_dep["id"]:
    response = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{dep}')
    data = response.json()
    data = data['dados']["ultimoStatus"]
    temp_df.append(data)

df_dep_rich = pd.DataFrame(temp_df)
df_dep_rich = df_dep_rich[["id","nome","siglaPartido","siglaUf","email","situacao","condicaoEleitoral"]]
df_dep_rich
    

In [6]:
temp_df = []
k = 1
for index, id_dep in enumerate(df_dep_rich["id"]):
    while True:
        url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/despesas?ano=2022&ano=2021&ano=2020&ano=2019&mes=12&mes=11&mes=10&mes=9&mes=8&mes=7&mes=6&mes=5&mes=4&mes=3&mes=2&mes=1&pagina={k}&itens=100&ordem=ASC&ordenarPor=ano'
        response = requests.get(url)
        data = response.json()
        data = data['dados']
        
        if len(data) == 0:
            break
        
        for i in range(len(data)):
            a = data[i]
            a['id'] = id_dep
            temp_df.append(a)
        
        k += 1

df_contas = pd.DataFrame(temp_df)


In [9]:
temp_df = []
k = 1
for index, id_dep in enumerate(df_dep_rich["id"]):
    while True:
        url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/discursos?dataInicio=2019-01-31&dataFim=2022-06-05&ordenarPor=dataHoraInicio&ordem=ASC&itens=100&pagina={k}'
        response = requests.get(url)
        data = response.json()
        data = data['dados']
        
        if len(data) == 0:
            break
        
        for i in range(len(data)):
            a = data[i]
            a['id'] = id_dep
            temp_df.append(a)
        
        k += 1

df_discursos = pd.DataFrame(temp_df)

In [ ]:
df_discursos.groupby(['tipoDiscurso']).count()

In [8]:
df_contas.to_csv('contas.csv', index=False, sep=',', decimal='.', encoding='utf-8')